In [1]:
## This file implements neural networks and random forest on p11presabs_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 90.85% for over-sampling data, 92.68% after improvement.
## And it is 98.11% for combination data and 95.28% with dropout.
## For random forest, the accuracy is 90.85% for over-sampling data and 99.06% for combination data.
## For random forest with cross-validation, the mean accuracy is 85.91% for over-sampling, and 96.75% for combination data.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p11presabs_qual.csv')
df.shape

(255, 974)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      2
1      1
2      2
3      2
4      2
5      2
6      2
7      2
8      2
9      2
10     1
11     2
12     2
13     1
14     2
15     2
16     0
17     0
18     2
19     0
20     0
21     0
22     1
23     2
24     2
25     0
26     2
27     0
28     2
29     2
      ..
225    2
226    1
227    2
228    2
229    2
230    1
231    2
232    2
233    2
234    2
235    2
236    2
237    2
238    2
239    2
240    2
241    2
242    0
243    2
244    2
245    2
246    2
247    2
248    2
249    2
250    2
251    1
252    2
253    2
254    2
Name: pheno, Length: 255, dtype: int64

In [6]:
df['pheno'].value_counts()

2    182
1     48
0     25
Name: pheno, dtype: int64

In [7]:
df.head()

,id,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,...,group_10073,group_10075,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2


In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 973)

In [10]:
df_clean.head()

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,TTTTTAGGTAAGG,...,group_10073,group_10075,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2


In [11]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 972) (255,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 182), (1, 182), (2, 182)]


Using TensorFlow backend.


In [13]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 164), (1, 111), (2, 77)]


In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
############# Fully-Connected Neural Network ################

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 382 samples, validate on 164 samples
Epoch 1/100
382/382 [==============================] - 0s 335us/step - loss: 1.2294 - accuracy: 0.4188 - val_loss: 0.9367 - val_accuracy: 0.6037
Epoch 2/100
382/382 [==============================] - 0s 70us/step - loss: 0.8102 - accuracy: 0.6466 - val_loss: 0.7446 - val_accuracy: 0.6159
Epoch 3/100
382/382 [==============================] - 0s 76us/step - loss: 0.7095 - accuracy: 0.6937 - val_loss: 0.6611 - val_accuracy: 0.7195
Epoch 4/100
382/382 [==============================] - 0s 75us/step - loss: 0.6588 - accuracy: 0.6990 - val_loss: 0.7022 - val_accuracy: 0.6463
Epoch 5/100
382/382 [==============================] - 0s 74us/step - loss: 0.6176 - accuracy: 0.7251 - val_loss: 0.6161 - val_accuracy: 0.7256
Epoch 6/100
382/382 [==============================] - 0s 82us/step - loss: 0.5772 - accuracy: 0.7618 - val_loss: 0.5778 - val_accuracy: 0.7866
Epoch 7/100
382/382 [==============================] - 0s 119us/step - loss: 0.5316 - acc

382/382 [==============================] - 0s 77us/step - loss: 0.1460 - accuracy: 0.9529 - val_loss: 0.3376 - val_accuracy: 0.9024
Epoch 58/100
382/382 [==============================] - 0s 75us/step - loss: 0.1430 - accuracy: 0.9634 - val_loss: 0.3371 - val_accuracy: 0.8598
Epoch 59/100
382/382 [==============================] - 0s 70us/step - loss: 0.1643 - accuracy: 0.9398 - val_loss: 0.3373 - val_accuracy: 0.8659
Epoch 60/100
382/382 [==============================] - 0s 69us/step - loss: 0.1520 - accuracy: 0.9372 - val_loss: 0.3492 - val_accuracy: 0.9024
Epoch 61/100
382/382 [==============================] - 0s 69us/step - loss: 0.1397 - accuracy: 0.9634 - val_loss: 0.3321 - val_accuracy: 0.9024
Epoch 62/100
382/382 [==============================] - 0s 75us/step - loss: 0.1460 - accuracy: 0.9529 - val_loss: 0.3395 - val_accuracy: 0.9024
Epoch 63/100
382/382 [==============================] - 0s 69us/step - loss: 0.1322 - accuracy: 0.9712 - val_loss: 0.3576 - val_accuracy: 0.884

In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

164/164 [==============================] - 0s 51us/step
over-sampling test accuracy: 90.85%


In [22]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [23]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 382 samples, validate on 164 samples
Epoch 1/1000
382/382 [==============================] - 0s 642us/step - loss: 3.7107 - accuracy: 0.4529 - val_loss: 0.8830 - val_accuracy: 0.6098
Epoch 2/1000
382/382 [==============================] - 0s 85us/step - loss: 3.6755 - accuracy: 0.4738 - val_loss: 0.8095 - val_accuracy: 0.6159
Epoch 3/1000
382/382 [==============================] - 0s 94us/step - loss: 3.2463 - accuracy: 0.5707 - val_loss: 0.7388 - val_accuracy: 0.6829
Epoch 4/1000
382/382 [==============================] - 0s 167us/step - loss: 2.8021 - accuracy: 0.6152 - val_loss: 0.7209 - val_accuracy: 0.6646
Epoch 5/1000
382/382 [==============================] - 0s 125us/step - loss: 3.3728 - accuracy: 0.5445 - val_loss: 0.8015 - val_accuracy: 0.6402
Epoch 6/1000
382/382 [==============================] - 0s 94us/step - loss: 3.3833 - accuracy: 0.5838 - val_loss: 0.7346 - val_accuracy: 0.7134
Epoch 7/1000
382/382 [==============================] - 0s 104us/step - loss: 3.1

Epoch 57/1000
382/382 [==============================] - 0s 184us/step - loss: 1.5394 - accuracy: 0.6885 - val_loss: 0.9278 - val_accuracy: 0.8354
Epoch 58/1000
382/382 [==============================] - 0s 165us/step - loss: 1.0788 - accuracy: 0.7435 - val_loss: 0.9555 - val_accuracy: 0.8049
Epoch 59/1000
382/382 [==============================] - 0s 134us/step - loss: 1.4136 - accuracy: 0.7199 - val_loss: 1.0205 - val_accuracy: 0.8171
Epoch 60/1000
382/382 [==============================] - 0s 138us/step - loss: 1.5470 - accuracy: 0.6937 - val_loss: 1.3444 - val_accuracy: 0.7500
Epoch 61/1000
382/382 [==============================] - 0s 139us/step - loss: 1.1867 - accuracy: 0.7461 - val_loss: 0.8913 - val_accuracy: 0.8780
Epoch 62/1000
382/382 [==============================] - 0s 141us/step - loss: 1.4211 - accuracy: 0.6990 - val_loss: 0.8622 - val_accuracy: 0.8049
Epoch 63/1000
382/382 [==============================] - 0s 99us/step - loss: 1.4561 - accuracy: 0.7251 - val_loss: 1.

Epoch 113/1000
382/382 [==============================] - 0s 109us/step - loss: 1.0141 - accuracy: 0.7435 - val_loss: 1.0179 - val_accuracy: 0.8476
Epoch 114/1000
382/382 [==============================] - 0s 120us/step - loss: 0.8059 - accuracy: 0.7618 - val_loss: 1.7140 - val_accuracy: 0.8476
Epoch 115/1000
382/382 [==============================] - 0s 135us/step - loss: 0.9095 - accuracy: 0.7304 - val_loss: 0.9831 - val_accuracy: 0.9085
Epoch 116/1000
382/382 [==============================] - 0s 133us/step - loss: 0.6973 - accuracy: 0.7880 - val_loss: 0.6943 - val_accuracy: 0.9146
Epoch 117/1000
382/382 [==============================] - 0s 156us/step - loss: 1.0011 - accuracy: 0.7539 - val_loss: 1.3776 - val_accuracy: 0.8598
Epoch 118/1000
382/382 [==============================] - 0s 233us/step - loss: 0.8729 - accuracy: 0.7251 - val_loss: 0.9936 - val_accuracy: 0.9024
Epoch 119/1000
382/382 [==============================] - 0s 134us/step - loss: 0.8074 - accuracy: 0.7801 - val_

382/382 [==============================] - 0s 252us/step - loss: 0.6728 - accuracy: 0.7723 - val_loss: 1.3419 - val_accuracy: 0.8415
Epoch 169/1000
382/382 [==============================] - 0s 212us/step - loss: 1.0476 - accuracy: 0.7592 - val_loss: 1.2423 - val_accuracy: 0.8598
Epoch 170/1000
382/382 [==============================] - 0s 200us/step - loss: 0.8419 - accuracy: 0.7251 - val_loss: 1.2716 - val_accuracy: 0.8537
Epoch 171/1000
382/382 [==============================] - 0s 222us/step - loss: 0.8118 - accuracy: 0.7513 - val_loss: 0.8155 - val_accuracy: 0.9085
Epoch 172/1000
382/382 [==============================] - 0s 167us/step - loss: 0.6038 - accuracy: 0.7775 - val_loss: 0.7664 - val_accuracy: 0.9268
Epoch 173/1000
382/382 [==============================] - 0s 105us/step - loss: 0.5473 - accuracy: 0.7775 - val_loss: 0.9714 - val_accuracy: 0.9085
Epoch 174/1000
382/382 [==============================] - 0s 110us/step - loss: 0.5518 - accuracy: 0.7853 - val_loss: 0.8736 - 

382/382 [==============================] - 0s 87us/step - loss: 0.6199 - accuracy: 0.7801 - val_loss: 1.0573 - val_accuracy: 0.8963
Epoch 224/1000
382/382 [==============================] - 0s 93us/step - loss: 0.4406 - accuracy: 0.7906 - val_loss: 0.9036 - val_accuracy: 0.9329
Epoch 225/1000
382/382 [==============================] - 0s 104us/step - loss: 0.4175 - accuracy: 0.8272 - val_loss: 1.0519 - val_accuracy: 0.8720
Epoch 226/1000
382/382 [==============================] - 0s 186us/step - loss: 0.6538 - accuracy: 0.7827 - val_loss: 0.8049 - val_accuracy: 0.9207
Epoch 227/1000
382/382 [==============================] - 0s 243us/step - loss: 0.3903 - accuracy: 0.8141 - val_loss: 0.8072 - val_accuracy: 0.9085
Epoch 228/1000
382/382 [==============================] - 0s 231us/step - loss: 0.5591 - accuracy: 0.8089 - val_loss: 0.8875 - val_accuracy: 0.9207
Epoch 229/1000
382/382 [==============================] - 0s 153us/step - loss: 0.4783 - accuracy: 0.7853 - val_loss: 0.9779 - va

...

382/382 [==============================] - 0s 107us/step - loss: 0.6083 - accuracy: 0.7539 - val_loss: 1.3820 - val_accuracy: 0.8963
Epoch 775/1000
382/382 [==============================] - 0s 106us/step - loss: 0.6030 - accuracy: 0.7749 - val_loss: 1.1034 - val_accuracy: 0.9268
Epoch 776/1000
382/382 [==============================] - 0s 98us/step - loss: 0.3732 - accuracy: 0.7749 - val_loss: 1.1068 - val_accuracy: 0.9207
Epoch 777/1000
382/382 [==============================] - 0s 100us/step - loss: 0.3658 - accuracy: 0.7853 - val_loss: 1.1366 - val_accuracy: 0.9268
Epoch 778/1000
382/382 [==============================] - 0s 112us/step - loss: 0.3423 - accuracy: 0.8089 - val_loss: 1.1400 - val_accuracy: 0.9268
Epoch 779/1000
382/382 [==============================] - 0s 150us/step - loss: 0.3853 - accuracy: 0.7932 - val_loss: 1.1395 - val_accuracy: 0.9268
Epoch 780/1000
382/382 [==============================] - 0s 111us/step - loss: 0.2771 - accuracy: 0.8298 - val_loss: 1.1372 - v

382/382 [==============================] - 0s 123us/step - loss: 0.3258 - accuracy: 0.8351 - val_loss: 0.9011 - val_accuracy: 0.9268
Epoch 830/1000
382/382 [==============================] - 0s 152us/step - loss: 0.5209 - accuracy: 0.7984 - val_loss: 1.2370 - val_accuracy: 0.9085
Epoch 831/1000
382/382 [==============================] - 0s 100us/step - loss: 0.3314 - accuracy: 0.8220 - val_loss: 1.0283 - val_accuracy: 0.9268
Epoch 832/1000
382/382 [==============================] - 0s 113us/step - loss: 0.4016 - accuracy: 0.7984 - val_loss: 1.1882 - val_accuracy: 0.9024
Epoch 833/1000
382/382 [==============================] - 0s 116us/step - loss: 0.4418 - accuracy: 0.7827 - val_loss: 1.1307 - val_accuracy: 0.9207
Epoch 834/1000
382/382 [==============================] - 0s 100us/step - loss: 0.4626 - accuracy: 0.8220 - val_loss: 1.0452 - val_accuracy: 0.9207
Epoch 835/1000
382/382 [==============================] - 0s 98us/step - loss: 0.4828 - accuracy: 0.7565 - val_loss: 1.1915 - v

Epoch 884/1000
382/382 [==============================] - 0s 135us/step - loss: 0.3241 - accuracy: 0.8246 - val_loss: 0.9828 - val_accuracy: 0.9329
Epoch 885/1000
382/382 [==============================] - 0s 110us/step - loss: 0.3713 - accuracy: 0.7958 - val_loss: 0.9556 - val_accuracy: 0.9329
Epoch 886/1000
382/382 [==============================] - 0s 107us/step - loss: 0.4078 - accuracy: 0.8010 - val_loss: 0.9935 - val_accuracy: 0.9268
Epoch 887/1000
382/382 [==============================] - 0s 97us/step - loss: 0.4206 - accuracy: 0.7670 - val_loss: 0.9812 - val_accuracy: 0.9390
Epoch 888/1000
382/382 [==============================] - 0s 107us/step - loss: 0.4169 - accuracy: 0.7906 - val_loss: 0.9966 - val_accuracy: 0.9268
Epoch 889/1000
382/382 [==============================] - 0s 115us/step - loss: 0.4766 - accuracy: 0.8220 - val_loss: 0.9436 - val_accuracy: 0.9329
Epoch 890/1000
382/382 [==============================] - 0s 112us/step - loss: 0.4247 - accuracy: 0.7958 - val_l

382/382 [==============================] - 0s 107us/step - loss: 0.3365 - accuracy: 0.7906 - val_loss: 1.1081 - val_accuracy: 0.9146
Epoch 940/1000
382/382 [==============================] - 0s 115us/step - loss: 0.4434 - accuracy: 0.8063 - val_loss: 1.0931 - val_accuracy: 0.9268
Epoch 941/1000
382/382 [==============================] - 0s 105us/step - loss: 0.3787 - accuracy: 0.8063 - val_loss: 1.0556 - val_accuracy: 0.9329
Epoch 942/1000
382/382 [==============================] - 0s 99us/step - loss: 0.3471 - accuracy: 0.8037 - val_loss: 1.0851 - val_accuracy: 0.9329
Epoch 943/1000
382/382 [==============================] - 0s 92us/step - loss: 0.3467 - accuracy: 0.7853 - val_loss: 1.0897 - val_accuracy: 0.9268
Epoch 944/1000
382/382 [==============================] - 0s 88us/step - loss: 0.3282 - accuracy: 0.7853 - val_loss: 1.0659 - val_accuracy: 0.9268
Epoch 945/1000
382/382 [==============================] - 0s 96us/step - loss: 0.3700 - accuracy: 0.7958 - val_loss: 1.0759 - val_

382/382 [==============================] - 0s 97us/step - loss: 0.4680 - accuracy: 0.7984 - val_loss: 0.9928 - val_accuracy: 0.9268
Epoch 995/1000
382/382 [==============================] - 0s 89us/step - loss: 0.3670 - accuracy: 0.7906 - val_loss: 0.9641 - val_accuracy: 0.9329
Epoch 996/1000
382/382 [==============================] - 0s 93us/step - loss: 0.3103 - accuracy: 0.7853 - val_loss: 0.9378 - val_accuracy: 0.9268
Epoch 997/1000
382/382 [==============================] - 0s 93us/step - loss: 0.3445 - accuracy: 0.8141 - val_loss: 1.0092 - val_accuracy: 0.9329
Epoch 998/1000
382/382 [==============================] - 0s 86us/step - loss: 0.4063 - accuracy: 0.7984 - val_loss: 0.9695 - val_accuracy: 0.9329
Epoch 999/1000
382/382 [==============================] - 0s 84us/step - loss: 0.3433 - accuracy: 0.7827 - val_loss: 0.9544 - val_accuracy: 0.9268
Epoch 1000/1000
382/382 [==============================] - 0s 85us/step - loss: 0.3881 - accuracy: 0.7565 - val_loss: 1.0211 - val_ac

In [25]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

164/164 [==============================] - 0s 78us/step
over-sampling test accuracy: 92.68%


In [26]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [27]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 246 samples, validate on 106 samples
Epoch 1/100
246/246 [==============================] - 0s 443us/step - loss: 1.0102 - accuracy: 0.4837 - val_loss: 0.6666 - val_accuracy: 0.7925
Epoch 2/100
246/246 [==============================] - 0s 83us/step - loss: 0.5679 - accuracy: 0.8089 - val_loss: 0.5049 - val_accuracy: 0.8585
Epoch 3/100
246/246 [==============================] - 0s 90us/step - loss: 0.4044 - accuracy: 0.8659 - val_loss: 0.4141 - val_accuracy: 0.8208
Epoch 4/100
246/246 [==============================] - 0s 86us/step - loss: 0.3222 - accuracy: 0.8699 - val_loss: 0.3335 - val_accuracy: 0.8962
Epoch 5/100
246/246 [==============================] - 0s 93us/step - loss: 0.2723 - accuracy: 0.9228 - val_loss: 0.3076 - val_accuracy: 0.8585
Epoch 6/100
246/246 [==============================] - 0s 96us/step - loss: 0.2474 - accuracy: 0.9106 - val_loss: 0.3014 - val_accuracy: 0.8585
Epoch 7/100
246/246 [==============================] - 0s 89us/step - loss: 0.2254 - accu

246/246 [==============================] - 0s 95us/step - loss: 0.0335 - accuracy: 0.9919 - val_loss: 0.1102 - val_accuracy: 0.9623
Epoch 58/100
246/246 [==============================] - 0s 97us/step - loss: 0.0289 - accuracy: 1.0000 - val_loss: 0.1112 - val_accuracy: 0.9717
Epoch 59/100
246/246 [==============================] - 0s 96us/step - loss: 0.0330 - accuracy: 1.0000 - val_loss: 0.1035 - val_accuracy: 0.9623
Epoch 60/100
246/246 [==============================] - 0s 93us/step - loss: 0.0333 - accuracy: 1.0000 - val_loss: 0.1143 - val_accuracy: 0.9717
Epoch 61/100
246/246 [==============================] - 0s 100us/step - loss: 0.0332 - accuracy: 1.0000 - val_loss: 0.1379 - val_accuracy: 0.9528
Epoch 62/100
246/246 [==============================] - 0s 102us/step - loss: 0.0354 - accuracy: 0.9959 - val_loss: 0.0967 - val_accuracy: 0.9623
Epoch 63/100
246/246 [==============================] - 0s 80us/step - loss: 0.0284 - accuracy: 1.0000 - val_loss: 0.1090 - val_accuracy: 0.9

In [29]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

106/106 [==============================] - 0s 136us/step
combination test accuracy: 98.11%


In [30]:
#### add dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [31]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 246 samples, validate on 106 samples
Epoch 1/100
246/246 [==============================] - 0s 691us/step - loss: 3.5771 - accuracy: 0.4837 - val_loss: 0.9080 - val_accuracy: 0.5755
Epoch 2/100
246/246 [==============================] - 0s 82us/step - loss: 3.9230 - accuracy: 0.5285 - val_loss: 0.6607 - val_accuracy: 0.7358
Epoch 3/100
246/246 [==============================] - 0s 99us/step - loss: 2.4786 - accuracy: 0.7033 - val_loss: 0.5433 - val_accuracy: 0.7830
Epoch 4/100
246/246 [==============================] - 0s 99us/step - loss: 3.2658 - accuracy: 0.6504 - val_loss: 0.4904 - val_accuracy: 0.7925
Epoch 5/100
246/246 [==============================] - 0s 100us/step - loss: 1.9874 - accuracy: 0.7602 - val_loss: 0.5111 - val_accuracy: 0.7736
Epoch 6/100
246/246 [==============================] - 0s 112us/step - loss: 2.7976 - accuracy: 0.6748 - val_loss: 0.4580 - val_accuracy: 0.7925
Epoch 7/100
246/246 [==============================] - 0s 114us/step - loss: 2.5870 - a

Epoch 57/100
246/246 [==============================] - 0s 142us/step - loss: 0.8386 - accuracy: 0.7398 - val_loss: 0.4640 - val_accuracy: 0.9340
Epoch 58/100
246/246 [==============================] - 0s 166us/step - loss: 0.6692 - accuracy: 0.7886 - val_loss: 0.3756 - val_accuracy: 0.9623
Epoch 59/100
246/246 [==============================] - 0s 170us/step - loss: 0.6853 - accuracy: 0.8049 - val_loss: 0.4472 - val_accuracy: 0.9434
Epoch 60/100
246/246 [==============================] - 0s 148us/step - loss: 0.8562 - accuracy: 0.7846 - val_loss: 0.3388 - val_accuracy: 0.9528
Epoch 61/100
246/246 [==============================] - 0s 130us/step - loss: 1.0456 - accuracy: 0.7439 - val_loss: 0.3114 - val_accuracy: 0.9340
Epoch 62/100
246/246 [==============================] - 0s 169us/step - loss: 0.7458 - accuracy: 0.7683 - val_loss: 0.4723 - val_accuracy: 0.8962
Epoch 63/100
246/246 [==============================] - 0s 154us/step - loss: 0.7967 - accuracy: 0.7805 - val_loss: 0.2977 -

In [33]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

106/106 [==============================] - 0s 126us/step
combination test accuracy: 95.28%


In [34]:
############## Random Forest ##############

In [35]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state = 123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [36]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 90.85%


In [39]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100,random_state = 123)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [40]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 99.06%


In [41]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.76923077 0.88461538 0.88157895 0.82666667 0.93333333]
0.8590850202429149


In [42]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.96       0.96       0.97959184 0.97959184 0.95833333]
0.9675034013605442
